In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from gensim.models import Word2Vec as wv

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
#import PhysicallyInformedLossFunction as PhysLoss



In [ ]:
#Opening contents of Word2Vec model1
data = '/Users/Thomas/Desktop/BETO2020-master/Ant_Syn_Scraping/all_abstracts_model'
os.chdir(data)
model1 = wv.load('all_abstract_model.model')
vocabulary1 = list(model1.wv.vocab)
#use model.build_vocab(sentence, update=True) to add missing words to model's vocabulary?
#or delete the rows that yield the KeyError?

In [ ]:
#Opening contents of Word2Vec model2
data = '/Users/Thomas/Desktop/BETO2020-master/Ant_Syn_Scraping/3j_15e_50D_min1_model'
os.chdir(data)
model2 = wv.load('3j_15e_50D_min1_model.model')
vocabulary2 = list(model2.wv.vocab)

In [ ]:
data = '/Users/Thomas/Desktop/BETO2020-master/Ant_Syn_Scraping/'
os.chdir(data)
carbon_df = pd.read_excel('Carbon_SynAntList_Full_Refined_copy.xlsx', skip_rows=1, nrows=2000, index_col=0)
carbon_df = carbon_df.rename(columns = {'Carbon_SynAntList_Full_Refined':'index', 'Unnamed: 1':'word 1', 'Unnamed: 2':'word 2','Unnamed: 3':'relationship', 'Unnamed: 4': 'label'})
carbon_df = carbon_df.fillna(0)
carbon_df = carbon_df[1:]

#finding which words are in the pd but not in vocabulary1
list1 = list(carbon_df['word 1'])
list2 = list(carbon_df['word 2'])
missing = list((set(list1).difference(vocabulary1))) + list((set(list2).difference(vocabulary1)))

#keeping only the rows in the pd that have words in vocabulary1
carbon_df = carbon_df[~carbon_df['word 1'].isin(missing)]
carbon_df = carbon_df[~carbon_df['word 2'].isin(missing)]

#reseting indeces after mask
carbon_df.reset_index(inplace = True)


In [ ]:
for i in range(len(carbon_df)):
    carbon_df['word 1'].iloc[i] = model1.wv.__getitem__(str(carbon_df['word 1'].iloc[i]))
    carbon_df['word 2'].iloc[i] = model1.wv.__getitem__(str(carbon_df['word 2'].iloc[i]))
    
    if carbon_df['relationship'].iloc[i] == 'syn' and carbon_df['label'].iloc[i] == 1:
        carbon_df['relationship'].iloc[i] = 1
    else:
        carbon_df['relationship'].iloc[i] = 0      
    

In [ ]:
carbon_df.to_json('SYN_NN_DATA.json')

In [ ]:
carbon_df = pd.read_json('SYN_NN_DATA.json', dtype = np.float32)

In [ ]:
#Hyper parameters
num_epochs = 100
batch_size = 50
learning_rate = 0.008

# Device configuration (GPU if available, otherwise CPU)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
X = carbon_df[['word 1', 'word 2']] #Input features used to make predictions
Y = carbon_df[['relationship']] #Target feature to be predicted 
 
x_train, x_test, syn_train, syn_test = train_test_split(X,Y, test_size = 0.2, shuffle = True) #split dataset into separate testing and training datasets

x_train.reset_index(inplace = True)
x_test.reset_index(inplace = True)
syn_train.reset_index(inplace = True)
syn_test.reset_index(inplace = True)

x1_train = x_train['word 1']
x2_train = x_train['word 2']
x1_test = x_test['word 1']
x2_test = x_test['word 2']

In [ ]:
#Had processed the data bit by bit instead of the whole x_train and x_test data sets

#x1_train_array = np.empty((len(x_train),50))
#x1_train_df = pd.DataFrame(data=x1_train_array[0:,:])

#x1_test_array = np.empty((len(x_test),50))
#x1_test_df = pd.DataFrame(data=x1_test_array[0:,:])

#x2_train_array = np.empty((len(x_train),50))
#x2_train_df = pd.DataFrame(data=x2_train_array[0:,:])

#x2_test_array = np.empty((len(x_test),50))
#x2_test_df = pd.DataFrame(data=x2_test_array[0:,:])

#for i in range(len(x1_train_df)):
#    for u in range(len(x_train['word 1'][i])):
#        x1_train_df.iloc[i,u] = x_train['word 1'][i][u]

#for i in range(len(x1_test_df)):
#    for u in range(len(x_test['word 1'][i])):
#        x1_test_df.iloc[i,u] = x_test['word 1'][i][u]
        
#for i in range(len(x2_train_df)):
#    for u in range(len(x_train['word 2'][i])):
#        x2_train_df.iloc[i,u] = x_train['word 2'][i][u]
        
#for i in range(len(x2_test_df)):
#    for u in range(len(x_test['word 2'][i])):
#        x2_test_df.iloc[i,u] = x_test['word 2'][i][u]
        

In [ ]:
x_train_array = np.empty((len(x_train),100))
x_train_df = pd.DataFrame(data=x_train_array[0:,:])

x_test_array = np.empty((len(x_test),100))
x_test_df = pd.DataFrame(data=x_test_array[0:,:])

In [ ]:
for i in range(len(x_train_df)):
    for u in range(len(x_train['word 2'][i])):
        x_train_df.iloc[i,u] = x_train['word 1'][i][u]
        
for i in range(len(x_train_df)):
    for u in range(len(x_train['word 2'][i])):
        x_train_df.iloc[i,(50+u)] = x_train['word 2'][i][u]

                
for i in range(len(x_test_df)):
    for u in range(len(x_test['word 1'][i])):
        x_test_df.iloc[i,u] = x_test['word 1'][i][u]
        
                
for i in range(len(x_test_df)):
    for u in range(len(x_test['word 2'][i])):
        x_test_df.iloc[i,(50+u)] = x_test['word 2'][i][u]


In [ ]:
x_train_tensor = torch.tensor(x_train_df.values.astype(np.float32)) #convert pd.DataFrame -> np.ndarray -> torch.tensor
syn_train_tensor = torch.tensor(syn_train.values.astype(np.float32))
#nonsyn_train_tensor = torch.tensor(nonsyn_train)

#create tensor with features and targets
train_tensor = torch.utils.data.TensorDataset(x_train_tensor, syn_train_tensor)
#create iterable dataset with batches
training_data_set = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)

x_test_tensor = torch.tensor(x_test_df.values.astype(np.float32))
syn_test_tensor = torch.tensor(syn_test.values.astype(np.float32))
#nonsyn_test_tensor = torch.tensor(nonsyn_test)

test_tensor = torch.utils.data.TensorDataset(x_test_tensor, syn_test_tensor)
testing_data_set = torch.utils.data.DataLoader(dataset = test_tensor, batch_size = batch_size, shuffle = True)


In [ ]:
class SYN_TEST(nn.Module):

    def __init__(self, in_dims, out_dims):
        super(SYN_TEST, self).__init__()
        
        #embedding layer
        self.embedding_layer = nn.Linear(in_dims, 100)
        
        #hidden layers
        self.hidden_layer = nn.Linear(100, 32)
        self.hidden_layer1 = nn.Linear(32, 16)
        
        #output layer
        self.output_layer = nn.Linear(16,2)
        #self.syn_branch = nn.Sequential(
        #nn.Dropout(p = 0.3),
        #nn.Linear(8,16),
        #nn.Linear(16,32),
        #nn.Linear(32,8),
        #nn.Dropout(p = 0.3)
        #nn.Softplus(),
        #nn.Linear(8,2))
        
    def forward(self,x):
       
        #pass through embedding layer
        out = self.embedding_layer(x)
        
        #pass through hidden layers
        out = self.hidden_layer(out)
        out = self.hidden_layer1(out)
        
        #pass to output layer
        syn_out = self.output_layer(out)
        
        return syn_out
    
        

In [ ]:
def train_model(model, training_data_set, optimizer):
    train_losses = []
    syn_train_losses = []
    train_epoch_loss = []
    syn_train_epoch_loss = []
    
    syn_losses = []
    train_total = 0
    
    #switch model to training mode
    model.train()
    syn_criterion = nn.MSELoss()
    
    for features, labels in training_data_set:
        
        model.zero_grad() #zero out any gradients from prior loops 
        syn_out = model(features) #gather model predictions for this loop
        
        #calculate error in the predictions
        syn_loss = syn_criterion(syn_out, labels)
        total_loss = syn_loss
        
        #BACKPROPAGATE LIKE A MF
        torch.autograd.backward([syn_loss])
        optimizer.step()
        
        #save loss for this batch
        train_losses.append(total_loss.item())
        train_total+=1
        
        syn_train_losses.append(syn_loss.item())
        
    #calculate and save total error for this epoch of training
    epoch_loss = sum(train_losses)/train_total
    train_epoch_loss.append(epoch_loss)
    
    syn_train_epoch_loss.append(sum(syn_train_losses)/train_total)
    
    #update progress bar
    print(f"Total Epoch Training Loss is: {train_epoch_loss}")
    
    return train_epoch_loss, syn_train_epoch_loss


In [ ]:
def eval_model(model, testing_data_set, optimizer):
    #evaluate the model
    model.eval()
    
    syn_criterion = nn.MSELoss()
    #accuracy = #total number of correct predictions divided by the total number of predictions

    #don't update nodes during evaluation b/c not training
    with torch.no_grad():
        test_losses = []
        syn_test_losses = []
        #syn_test_acc_list = []
        
        test_total = 0

        for inputs, labels in testing_data_set:
            inputs = inputs.to(device)
            labels = labels.to(device)

            syn_out = model(inputs)

            # calculate loss per batch of testing data
            syn_test_loss = syn_criterion(syn_out, labels)
            
            test_loss = syn_test_loss
            
            test_losses.append(test_loss.item())
            syn_test_losses.append(syn_test_loss.item())
            test_total += 1 
            #syn_acc = accuracy(syn_out)
            #syn_test_acc_list.append(syn_acc.item())

        test_epoch_loss = sum(test_losses)/test_total
        syn_test_epoch_loss = sum(syn_test_losses)/test_total
        
        #syn_epoch_acc = sum(syn_test_acc_list)/test_total

        print(f"Total Epoch Testing Loss is: {test_epoch_loss}")
        #print(f"Epoch MAPE: Syn = {syn_epoch_acc}")
    
    return test_epoch_loss, syn_test_epoch_loss, #syn_epoch_acc




In [ ]:
# Instantiate our beautiful NN model
# takes in 
# predicts synonymy
model = SYN_TEST(in_dims = 100, out_dims = 2).to(device)

#define the optimizer
optimizer = torch.optim.Adam(params = model.parameters(), lr = learning_rate)



In [ ]:
#empty list to hold loss per epoch
train_epoch_losses = []
syn_train_epoch_losses = []

test_epoch_losses = []
syn_test_epoch_losses = []

syn_test_epoch_accuracies = []


for epoch in range(num_epochs):
    
    train_epoch_loss, syn_train_epoch_loss  = train_model(model = model, training_data_set = training_data_set, optimizer = optimizer)
    
    train_epoch_losses.append(train_epoch_loss)
    syn_train_epoch_losses.append(syn_train_epoch_loss)
   
    test_epoch_loss, syn_test_epoch_loss = eval_model(model = model, testing_data_set = testing_data_set, optimizer = optimizer)
    #syn_epoch_acc
    
    test_epoch_losses.append(test_epoch_loss)
    syn_test_epoch_losses.append(syn_test_epoch_loss)
    
    #pce_test_epoch_accuracies.append(pce_epoch_acc)
    #voc_test_epoch_accuracies.append(voc_epoch_acc)
    #jsc_test_epoch_accuracies.append(jsc_epoch_acc)
    #ff_test_epoch_accuracies.append(ff_epoch_acc)
    

In [ ]:
fig, ax = plt.subplots(figsize = (8,6))

epochs = np.arange(1, (num_epochs+1), 1)

plt.plot(epochs, train_epoch_losses, c = 'k', label = 'training error')
plt.plot(epochs, test_epoch_losses, c = 'r', label = 'testing error')
plt.legend(loc = 'upper right')
plt.title("Total Training & Testing Error")
ax.set_xlabel('Epoch')
ax.set_ylabel('Total MSE Loss')
plt.show()

#fig, ax = plt.subplots(figsize = (8,6))
# plt.plot(epochs, train_epoch_accuracy, c = 'k', label = 'training accuracy')
#plt.plot(epochs, pce_test_epoch_accuracies, c = 'k', label = 'pce accuracy')
#plt.plot(epochs, voc_test_epoch_accuracies, c = 'r', label = 'voc accuracy')
#plt.plot(epochs, jsc_test_epoch_accuracies, c = 'g', label = 'jsc accuracy')
#plt.plot(epochs, ff_test_epoch_accuracies, c = 'b', label = 'ff accuracy')
#plt.legend(loc = 'lower right')
#plt.title("Branch Mean Absolute Percent Error")
#ax.set_xlabel('Epoch')
#ax.set_ylabel('MAPE')
#plt.show()

In [ ]:
xlin = ylin = np.arange(0, 100, 1)

fig, ax = plt.subplots(figsize = (8,6))

plt.scatter(PCE_out, pce_labels)
plt.plot(xlin, ylin, c = 'k')
ax.set_xlim(0, 3)
ax.set_ylim(0, 3)
ax.set_xlabel("Predictions")
ax.set_ylabel("Ground Truth")
plt.show()

fig, ax = plt.subplots(figsize = (8,6))

plt.scatter(Voc_out, voc_labels)
plt.plot(xlin, ylin, c = 'k')
ax.set_xlim(0, 0.9)
ax.set_ylim(0, 0.9)
ax.set_xlabel("Predictions")
ax.set_ylabel("Ground Truth")
plt.show()

fig, ax = plt.subplots(figsize = (8,6))

plt.scatter(Jsc_out, jsc_labels)
plt.plot(xlin, ylin, c = 'k')
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.set_xlabel("Predictions")
ax.set_ylabel("Ground Truth")
plt.show()

fig, ax = plt.subplots(figsize = (8,6))

plt.scatter(FF_out, ff_labels)
plt.plot(xlin, ylin, c = 'k')
ax.set_xlim(0, 100)
ax.set_ylim(0, 100)
ax.set_xlabel("Predictions")
ax.set_ylabel("Ground Truth")
plt.show()
